In [1]:
using DrWatson
@quickactivate

In [2]:
using BenchmarkTools, StaticArrays
import ReverseDiff, Tracker, Zygote
const GRADS = (
    "ReverseDiff" => ReverseDiff.gradient, 
    "Tracker" => Tracker.gradient, 
    "Zygote" => Zygote.gradient,
)

;

In [3]:
function logdensity(x)
    res = 0
    for i = 1:size(x, 1)
        res += x[i]
    end
    return res
end

macro unroll_logdensity(n)
    ex = :(function $(Symbol("logdensity_$n"))(x) end)
    body = ex.args[2]
    push!(body.args, :(res = 0))
    for i in 1:n
        push!(body.args, :(res += x[$i]))
    end
    push!(body.args, :(return res))
    ex
end

@macroexpand(@unroll_logdensity(20)) |> display

;

:(function Main.logdensity_20(var"#10#x")
      #= In[3]:10 =#
      var"#9#res" = 0
      var"#9#res" += var"#10#x"[1]
      var"#9#res" += var"#10#x"[2]
      var"#9#res" += var"#10#x"[3]
      var"#9#res" += var"#10#x"[4]
      var"#9#res" += var"#10#x"[5]
      var"#9#res" += var"#10#x"[6]
      var"#9#res" += var"#10#x"[7]
      var"#9#res" += var"#10#x"[8]
      var"#9#res" += var"#10#x"[9]
      var"#9#res" += var"#10#x"[10]
      var"#9#res" += var"#10#x"[11]
      var"#9#res" += var"#10#x"[12]
      var"#9#res" += var"#10#x"[13]
      var"#9#res" += var"#10#x"[14]
      var"#9#res" += var"#10#x"[15]
      var"#9#res" += var"#10#x"[16]
      var"#9#res" += var"#10#x"[17]
      var"#9#res" += var"#10#x"[18]
      var"#9#res" += var"#10#x"[19]
      var"#9#res" += var"#10#x"[20]
      return var"#9#res"
  end)

In [4]:
function trybtime(f, x; verbose=true)
    try
        t = @belapsed $f($x)
        verbose && println(" --- takes $t seconds")
        return t
    catch e
        verbose && println(" --- fails due to $e")
        return NaN
    end
end

function benchmark(f, x0; verbose=true)
    xs = [x0, SVector{length(x0), eltype(x0)}(x0)]
    res = []
    for x in xs
        verbose && print(rpad("$f(x::$(typeof(x)))", 55))
        trybtime(f, x; verbose=verbose)
        x isa SVector && continue
        ts = []
        for grad in GRADS
            gradn, gradf = grad
            verbose && print("- $(lpad(gradn, 12)).gradient")
            t = trybtime(x -> gradf(f, x), x; verbose=verbose)
            push!(ts, t)
        end
        push!(res, ts)
    end
    return res
end

;

In [5]:
let x = rand(500)
    benchmark(logdensity, x)
    benchmark(@unroll_logdensity(500), x)
end

;

logdensity(x::Array{Float64,1})                         --- takes 5.889401197604791e-7 seconds
-  ReverseDiff.gradient --- takes 0.000106365 seconds
-      Tracker.gradient --- takes 0.000342397 seconds
-       Zygote.gradient --- takes 0.001322017 seconds
logdensity(x::SArray{Tuple{500},Float64,1,500})         --- takes 4.517817258883249e-7 seconds
logdensity_500(x::Array{Float64,1})                     --- takes 5.177552083333334e-7 seconds
-  ReverseDiff.gradient --- takes 0.000117554 seconds
-      Tracker.gradient --- takes 0.000376928 seconds
-       Zygote.gradient --- takes 0.003709555 seconds
logdensity_500(x::SArray{Tuple{500},Float64,1,500})     --- takes 3.6e-11 seconds


In [6]:
let x = rand(1_000)
    benchmark(logdensity, x)
    benchmark(@unroll_logdensity(1_000), x)
end

;

logdensity(x::Array{Float64,1})                         --- takes 1.0970999999999998e-6 seconds
-  ReverseDiff.gradient --- takes 0.000214341 seconds
-      Tracker.gradient --- takes 0.001070315 seconds
-       Zygote.gradient --- takes 0.003905595 seconds
logdensity(x::SArray{Tuple{1000},Float64,1,1000})       --- takes 8.815294117647058e-7 seconds
logdensity_1000(x::Array{Float64,1})                    --- takes 1.025e-6 seconds
-  ReverseDiff.gradient --- takes 0.000250316 seconds
-      Tracker.gradient --- takes 0.001164586 seconds
-       Zygote.gradient --- takes 0.017711017 seconds
logdensity_1000(x::SArray{Tuple{1000},Float64,1,1000})  --- takes 3.6999999999999995e-11 seconds
